<h2><br>WDB Systems India Pvt Ltd</br>
<br>Member 1:- KOSURI PRAHARSHA</br>
<br>Member 2:- KOSURI VENU MADHAVA RAO </br>
<br>SPECTRA PURE –MODEL</br>


<h2> Import Libraries </h2>

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import scipy.optimize as op


<h2>Configuration </h2>

In [23]:
input_file="heart_data.csv"
split_percentage=0.3

#Model Tuning Paramaeters
#alpha = 0.005
alpha = 0.01
#epochs = 150000
epochs = 15000
batch_size= 16
lambda_reg = 0.01

In [24]:
hrt_df = pd.read_csv(input_file)
hrt_df.head(0)
hrt_df.shape 

# Drop duplicates
hrt_df.drop_duplicates(inplace=True)

# Remove outliers using IQR (only for continuous features)
col_num = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
for col in col_num:
    Q1 = hrt_df[col].quantile(0.25)
    Q3 = hrt_df[col].quantile(0.75)
    IQR = Q3 - Q1
    LowerBound = Q1 - 1.5 * IQR
    UpperBound = Q3 + 1.5 * IQR
    hrt_df = hrt_df[(hrt_df[col] >= LowerBound) & (hrt_df[col] <= UpperBound)]

hrt_df = hrt_df.reset_index(drop=True)


<h2>Extract input (X) and output (Y)</h2>

In [25]:

# Split features and target
X = hrt_df.drop(columns=['target']).values
print('i am here')
print(X[0]);
Y = hrt_df['target'].values.reshape(-1, 1) 
print(f"✅ Data Shape: {X.shape}")

i am here
[ 63.    1.    3.  145.  233.    1.    0.  150.    0.    2.3   0.    0.
   1. ]
✅ Data Shape: (283, 13)


<h2>Splitting training and test data</h2>

In [5]:
# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=split_percentage, random_state=20)
print(f"✅ Data Ready. Train: {X_train.shape}, Test: {X_test.shape}")
'''
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)
'''
scaler = MinMaxScaler()#It is normalisation of features.
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Y_train = Y_train.reshape(-1,1)
Y_test = Y_test.reshape(-1,1)



✅ Data Ready. Train: (198, 13), Test: (85, 13)


In [26]:

X_new = X_test[-1].reshape(1, -1)
Y_new = Y_test[-1]

X_test = X_test[:-1]
Y_test = Y_test[:-1]

<h2>Helper Functions</h2>

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(y, y_pred):
    m = len(y)
    epsilon = 1e-8
    return -(1/m) * np.sum(y*np.log(y_pred + epsilon) + (1-y)*np.log(1 - y_pred + epsilon))

def f1_score_func(y_true, y_pred, threshold=0.5):
    y_pred_class = (y_pred >= threshold).astype(int)
    tp = np.sum((y_true == 1) & (y_pred_class == 1))
    fp = np.sum((y_true == 0) & (y_pred_class == 1))
    fn = np.sum((y_true == 1) & (y_pred_class == 0))
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

def plot_loss(loss_history, title):
    plt.figure(figsize=(7,5))
    plt.plot(loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(title)
    plt.show()
    
def evaluate_model(weights, bias, X_test, Y_test, title):
    y_pred_probs = sigmoid(np.dot(X_test, weights) + bias)
    y_pred_class = (y_pred_probs >= 0.5).astype(int)

    accuracy = accuracy_score(Y_test, y_pred_class)
    report = classification_report(Y_test, y_pred_class, output_dict=True, zero_division=0)
    conf_mat = confusion_matrix(Y_test, y_pred_class)
    f1 = f1_score_func(Y_test, y_pred_probs)

    tn, fp, fn, tp = conf_mat.ravel()
    specificity = tn / (tn + fp + 1e-8)
    sensitivity = tp / (tp + fn + 1e-8)

    print(f"\n📊 {title} Results:")
    print("Accuracy:", accuracy)
    print("F1-Score:", f1)
    print("Sensitivity (Recall):", sensitivity)
    print("Specificity:", specificity)
    print("Confusion Matrix:\n", conf_mat)

    return {
        "Method": title,
        "Accuracy": accuracy,
        "F1-Score": f1,
        "Sensitivity": sensitivity,
        "Specificity": specificity
    }


<h2>Vanilla/Full Batch Gradient Descent</h2>

In [8]:
def batch_gradient_descent(X, Y, alpha, epochs):
    n_samples, n_features = X.shape
    weights = np.zeros((n_features, 1))
    bias = 0
    loss_history = []

    for i in range(epochs):
        z = np.dot(X, weights) + bias
        y_pred = sigmoid(z)

        dw = (1/n_samples) * np.dot(X.T, (y_pred - Y))
        db = (1/n_samples) * np.sum(y_pred - Y)

        weights -= alpha * dw
        bias -= alpha * db

        loss = compute_loss(Y, y_pred)
        loss_history.append(loss)

    print("\n🔹 Final Weights (Batch GD):", weights.ravel())
    print("🔹 Final Bias (Batch GD):", bias)

    return weights, bias, loss_history

<h2>Stoichastic Gradient Descent</h2>

In [9]:
def stochastic_gradient_descent(X, Y, alpha, epochs):
    n_samples, n_features = X.shape
    weights = np.zeros((n_features, 1))
    bias = 0
    loss_history = []

    for epoch in range(epochs):
        indices = np.arange(n_samples)
        np.random.shuffle(indices)

        for i in indices:
            x_i = X[i].reshape(1, -1)
            y_i = Y[i]

            z = np.dot(x_i, weights) + bias
            y_pred = sigmoid(z)

            dw = np.dot(x_i.T, (y_pred - y_i))
            db = (y_pred - y_i)

            weights -= alpha * dw
            bias -= alpha * db

        y_pred_all = sigmoid(np.dot(X, weights) + bias)
        loss = compute_loss(Y, y_pred_all)
        loss_history.append(loss)

    print("\n🔹 Final Weights (Stochastic GD):", weights.ravel())
    print("🔹 Final Bias (Stochastic GD):", bias)

    return weights, bias, loss_history


<h2>Minibatch Gradient Descent</h2>

In [10]:
def mini_batch_gradient_descent(X, Y, alpha, epochs, batch_size):
    n_samples, n_features = X.shape
    weights = np.zeros((n_features, 1))
    bias = 0
    loss_history = []

    for epoch in range(epochs):
        indices = np.arange(n_samples)
        #print("indices: ",indices)
        np.random.shuffle(indices)
        X_s = X[indices]
        Y_s = Y[indices]

        for start in range(0, n_samples, batch_size):
            end = start + batch_size
            x_batch = X_s[start:end]
            y_batch = Y_s[start:end]

            z = np.dot(x_batch, weights) + bias
            y_pred = sigmoid(z)

            dw = (1 / len(x_batch)) * np.dot(x_batch.T, (y_pred - y_batch))
            db = (1 / len(x_batch)) * np.sum(y_pred - y_batch)

            weights -= alpha * dw
            bias -= alpha * db

        y_pred_all = sigmoid(np.dot(X, weights) + bias)
        loss = compute_loss(Y, y_pred_all)
        loss_history.append(loss)

    print("\n🔹 Final Weights (Mini-Batch GD):", weights.ravel())
    print("🔹 Final Bias (Mini-Batch GD):", bias)

    return weights, bias, loss_history


<h2>Vanilla2 Gradient Descent</h2>

In [11]:
# --------------------------
# 5️⃣+ Vanilla Logistic Regression (using scipy.optimize)
# --------------------------

def vanilla2(X, Y):
    import scipy.optimize as op

    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    def costFunction(theta, X, y):
        m, n = X.shape
        theta = theta.reshape((n, 1))
        z = np.dot(X, theta)
        h = sigmoid(z)
        epsilon = 1e-8
        cost = -(1/m) * np.sum(y*np.log(h+epsilon) + (1-y)*np.log(1-h+epsilon))
        grad = (1/m) * np.dot(X.T, (h - y))
        return cost, grad.flatten()

    # Add bias column (intercept term)
    X_bias = np.hstack((np.ones((X.shape[0], 1)), X))
    m, n = X_bias.shape
    initial_theta = np.zeros((n, 1))

    # Optimize using scipy fmin_tnc
    result = op.fmin_tnc(func=costFunction, x0=initial_theta, args=(X_bias, Y))
    theta_opt = result[0].reshape((n, 1))

    weights = theta_opt[1:]
    bias = theta_opt[0]

    print("\n🔹 Final Weights (Vanilla2):", weights.ravel())
    print("🔹 Final Bias (Vanilla2):", bias)

    return weights, bias


<h2>Training and Evaluating Models</h2>

In [12]:
results = []
'''
# Batch
w_b, b_b, loss_b = batch_gradient_descent(X_train, Y_train,alpha,epochs)
plot_loss(loss_b, "Batch Gradient Descent Loss")
results.append(evaluate_model(w_b, b_b, X_test, Y_test, "Batch Gradient Descent"))
'''

'\n# Batch\nw_b, b_b, loss_b = batch_gradient_descent(X_train, Y_train,alpha,epochs)\nplot_loss(loss_b, "Batch Gradient Descent Loss")\nresults.append(evaluate_model(w_b, b_b, X_test, Y_test, "Batch Gradient Descent"))\n'

In [13]:
'''
# Stochastic
w_s, b_s, loss_s = stochastic_gradient_descent(X_train, Y_train,alpha,epochs)
plot_loss(loss_s, "Stochastic Gradient Descent Loss")
results.append(evaluate_model(w_s, b_s, X_test, Y_test, "Stochastic Gradient Descent"))
'''


'\n# Stochastic\nw_s, b_s, loss_s = stochastic_gradient_descent(X_train, Y_train,alpha,epochs)\nplot_loss(loss_s, "Stochastic Gradient Descent Loss")\nresults.append(evaluate_model(w_s, b_s, X_test, Y_test, "Stochastic Gradient Descent"))\n'

In [14]:
'''
# Mini-Batch
w_m, b_m, loss_m = mini_batch_gradient_descent(X_train, Y_train,alpha,epochs,batch_size)
plot_loss(loss_m, "Mini-Batch Gradient Descent Loss")
results.append(evaluate_model(w_m, b_m, X_test, Y_test, "Mini-Batch Gradient Descent"))
'''

'\n# Mini-Batch\nw_m, b_m, loss_m = mini_batch_gradient_descent(X_train, Y_train,alpha,epochs,batch_size)\nplot_loss(loss_m, "Mini-Batch Gradient Descent Loss")\nresults.append(evaluate_model(w_m, b_m, X_test, Y_test, "Mini-Batch Gradient Descent"))\n'

In [15]:
'''
# Vanilla2 (using scipy.optimize)
w_v2, b_v2 = vanilla2(X_train, Y_train)
results.append(evaluate_model(w_v2, b_v2, X_test, Y_test, "Vanilla2 Logistic Regression"))
'''

'\n# Vanilla2 (using scipy.optimize)\nw_v2, b_v2 = vanilla2(X_train, Y_train)\nresults.append(evaluate_model(w_v2, b_v2, X_test, Y_test, "Vanilla2 Logistic Regression"))\n'

<h2>Summary Table</h2>

In [16]:
summary_df = pd.DataFrame(results)
print("\n✅ Final Performance Summary:")
print(summary_df)


✅ Final Performance Summary:
Empty DataFrame
Columns: []
Index: []


<h2>Inference Cell</h2>

In [17]:
def inference(X_new, weights, bias):
    y_predict = sigmoid(np.dot(X_new, weights) + bias)
    #print(f"Input values given = {X_new},\nWeights = {weights},\nBias = {bias}")
    if(y_predict>0.5):
        return 1
    else:
        return 0
'''
print("Calling Vanilla Gradient Inference")
Y_output = inference(X_new, w_b, b_b)
print(f"Predicted value using vanilla gradient = {Y_output}, actual value = {Y_new}")
#print(f"X= {X_new}",X_new)
print("Calling Stoichastic Gradient Inference")
Y_output = inference(X_new, w_s, b_s)
print(f"Predicted value using stoichastic gradient = {Y_output}, actual value = {Y_new}")

print("Calling Mini Gradient Inference")
Y_output = inference(X_new, w_m, b_m)
print(f"Predicted value using mini gradient = {Y_output}, actual value = {Y_new}")

print("Calling Vanilla2 Gradient Inference")
Y_output = inference(X_new, w_v2, b_v2)
print(f"Predicted value using vanilla2 gradient = {Y_output}, actual value = {Y_new}")
'''

'\nprint("Calling Vanilla Gradient Inference")\nY_output = inference(X_new, w_b, b_b)\nprint(f"Predicted value using vanilla gradient = {Y_output}, actual value = {Y_new}")\n#print(f"X= {X_new}",X_new)\nprint("Calling Stoichastic Gradient Inference")\nY_output = inference(X_new, w_s, b_s)\nprint(f"Predicted value using stoichastic gradient = {Y_output}, actual value = {Y_new}")\n\nprint("Calling Mini Gradient Inference")\nY_output = inference(X_new, w_m, b_m)\nprint(f"Predicted value using mini gradient = {Y_output}, actual value = {Y_new}")\n\nprint("Calling Vanilla2 Gradient Inference")\nY_output = inference(X_new, w_v2, b_v2)\nprint(f"Predicted value using vanilla2 gradient = {Y_output}, actual value = {Y_new}")\n'

<h2>Built In Function(SVM)</h2>

In [59]:
def svm_linear(X, Y, lambda_reg=0.01): 
    # Ensure Y is 1D for sklearn
    Y_sklearn = Y.ravel()  # converts (n,1) to (n,)  
    model = LinearSVC(C=1/lambda_reg, max_iter=10000, random_state=20)      
    model.fit(X, Y_sklearn)
    return model.coef_.T,model.intercept_[0]
  
def svm_nonlinear(X, Y, kernel_type="linear", lambda_reg=0.01, kernel_name="rbf"):
    Y_sklearn = Y.ravel()
    model = SVC(kernel=kernel_name, C=1/lambda_reg, gamma='scale', random_state=20)
    model.fit(X, Y_sklearn)
    print(model.support_vectors_)
    print(model.dual_coef_)   #Alpha_i*Y_i
    print(model.intercept_)  #B
    return model

def svm_linear_inference(X,weights,bias):
    result=np.dot(X,weights) + bias
    print(result)
    if (result>0):
        return 1
    else:
        return 0

def svm_nonlinear_inference(X,model):
    X=X.T
    X=X.reshape(1,-1)
    return model.predict(X)
    
'''
weights_svm,bias_svm=svm_linear(X_train,Y_train,0.01)
#print(f"weights shape: {weights_svm.shape},bias shape: {bias_svm.shape}",weights_svm.shape,bias_svm.shape)
Y_output_svm=svm_linear_inference(X_new,weights_svm,bias_svm)
print(f"Predicted value using linear svm= {Y_output_svm}, actual value = {Y_new}")
X_new=X_test[0]
Y_new=Y_test[0]
print(X_new)
print(Y_new)
Y_output_svm=svm_linear_inference(X_new,weights_svm,bias_svm)
print(f"Predicted value using linear svm= {Y_output_svm}, actual value = {Y_new}")
'''

svm_model_nonlinear=svm_nonlinear(X_train, Y_train, kernel_type="linear", lambda_reg=0.01, kernel_name="rbf")
Y_output_svm=svm_nonlinear_inference(X_new,svm_model_nonlinear)
print(f"Predicted value using non linear svm= {Y_output_svm}, actual value = {Y_new}")


[[0.29787234 1.         0.         ... 0.5        0.         1.        ]
 [0.61702128 1.         0.66666667 ... 1.         0.5        1.        ]
 [0.59574468 1.         0.         ... 0.5        0.25       0.33333333]
 ...
 [0.4893617  1.         0.33333333 ... 1.         0.         0.66666667]
 [0.34042553 1.         0.         ... 0.5        0.         0.66666667]
 [0.46808511 1.         0.         ... 1.         0.         0.66666667]]
[[-9.05726101e+00 -4.73206037e+00 -2.12506625e+00 -1.76718884e+00
  -1.61204019e+01 -9.79340000e+00 -9.80632051e-01 -3.41999481e+01
  -9.86009873e+01 -7.77842834e+00 -5.75035023e+01 -1.37476693e+00
  -1.23119097e+00 -1.94504851e+01 -6.70668327e+01 -1.33556502e+01
  -2.29155410e+01 -3.84955811e+00 -1.21669891e+01 -2.24577187e+01
  -1.99771339e+01 -1.77725020e+01 -1.85682306e+01 -6.42029134e-01
  -8.69245832e-01 -3.57302761e+00 -9.62968457e-01 -2.98265213e+01
  -6.38482688e+00 -3.27326327e+01 -2.32158508e+01 -1.29424471e+01
  -5.38200641e+00 -2.2482087